# Polynomial Regression
So according to the visualizations from the <a href="https://github.com/lynstanford/machine-learning-projects/machine-learning/multiple_regression.ipynb">Linear Regression</a> notebook, the connection between the dependent and independent variables appear to be strongly linear in relationship although the connection between the 'Volume' of daily bitcoin bought and sold has less association with the daily 'Close' price. 

Fitting a Linear Regression line to the data may be accurate in this case, with an R2 value of 0.9991392014437468 and RMSE of 689.1925598643533. However, out of curiosity I decided to see if a Polynomial function could fit the line slightly better by employing a regularization technique to try and improve the bias term by decreasing the Mean Squared Error.

The r-squared value is used to represent the overall accuracy score and directly measures the degree of variability associated between the predictors and target variable. The root mean squared value is represented as a loss function and my aim is to reduce its overall value as much as possible using regularization.

## Import Data
Keeping the data loading simple this time will reduce the overall time it takes to retrieve.

In [1]:
# import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# import data from filepath
btc_cad = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

# storing dataset from filepath into new dataframe
bitcoin = pd.DataFrame(btc_cad).dropna(axis=0)

This provides the entire dataset with all the values stored in a single dataframe.

## Feature Selection and Scaling
Checking to see what features are present within the current dataframe:

In [2]:
# all column names
bitcoin.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

Now to remove the columns I don't need including 'Date' and 'Adj Close', selecting only the remaining ones to include in the X variable such as 'Open', 'High', 'Low' and 'Volume'. The remaining column of 'Close' in the same dataframe will be used as the target variable output, y.

In [3]:
# remove feature with string values - Date
del bitcoin['Date']

# remove adj close as I will not be using this
del bitcoin['Adj Close']

# see the remaining features
bitcoin.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

Assigning features to X gives:

In [4]:
# select features as X dataframe
X = bitcoin[['Open','High','Low','Volume']]
print(X)

         Open      High       Low        Volume
0     9718.07   9838.33   9728.25  4.624843e+10
1     9762.68  10125.87   9747.39  4.716635e+10
2    10102.09  10533.73  10009.76  6.119120e+10
3    10457.43  10678.71  10457.43  4.881932e+10
4    10642.22  10773.18  10601.61  4.643028e+10
..        ...       ...       ...           ...
360  79368.23  79831.47  75262.20  1.050000e+11
361  76964.70  78268.41  75503.00  8.272967e+10
362  75928.95  76373.72  66081.83  1.220000e+11
363  70344.11  71803.61  68107.27  8.183693e+10
365  71509.30  71556.76  71132.08  8.495634e+10

[362 rows x 4 columns]


Comparing the shape of the overall dataset before polynomial regression and before splitting gives:

In [5]:
X.shape

(362, 4)

In [6]:
# select target as y series
y = bitcoin[['Close']]
print(y)

        Close
0     9763.94
1    10096.28
2    10451.16
3    10642.81
4    10669.64
..        ...
360  77018.32
361  75906.36
362  70374.91
363  69788.23
365  71477.76

[362 rows x 1 columns]


In [7]:
y.shape

(362, 1)

Another way to select the right column vectors is using indexation.

In [8]:
X = X.iloc[:,0:4].values
print(X[0:10])

[[9.71807000e+03 9.83833000e+03 9.72825000e+03 4.62484281e+10]
 [9.76268000e+03 1.01258700e+04 9.74739000e+03 4.71663519e+10]
 [1.01020900e+04 1.05337300e+04 1.00097600e+04 6.11912020e+10]
 [1.04574300e+04 1.06787100e+04 1.04574300e+04 4.88193179e+10]
 [1.06422200e+04 1.07731800e+04 1.06016100e+04 4.64302791e+10]
 [1.06699200e+04 1.08566800e+04 1.06576200e+04 4.66636415e+10]
 [1.08360400e+04 1.09416000e+04 1.08360400e+04 5.07557760e+10]
 [1.09435200e+04 1.09270500e+04 1.08227000e+04 4.64088816e+10]
 [1.09166300e+04 1.23147700e+04 1.08593000e+04 8.35291405e+10]
 [1.22067900e+04 1.31051300e+04 1.19009200e+04 9.34508816e+10]]


In [9]:
y = y.iloc[:, ].values
print(y[0:10])

[[ 9763.94]
 [10096.28]
 [10451.16]
 [10642.81]
 [10669.64]
 [10836.68]
 [10941.6 ]
 [10917.12]
 [12211.47]
 [12083.24]]


Does the dataset need re-scaling? In this particular case the (X) predictors include 3 price variables and 1 volume variable which is scaled differently. The (y) target variable is another price variable, so the data would benefit from re-scaling. The transformation I have chosen will shift the values to a range between 0 and 1 for each column. 

Make a copy of the dataframe first.

In [10]:
bitcoin = bitcoin.copy()

In [18]:
# import preprocessing from sci-kit learn
from sklearn import preprocessing

# define min max scaler
min_max_scaler = preprocessing.MinMaxScaler()

# transform data
X_scaled = min_max_scaler.fit_transform(X)

bitcoin_features = pd.DataFrame(X_scaled)

bitcoin_features.to_csv(r'C:\Users\lynst\Documents\GitHub\machine-learning-projects\machine-learning\bitcoin_features.csv', index = False, header = True)

print(bitcoin_features)

            0         1         2         3
0    0.000000  0.000000  0.000000  0.068801
1    0.000638  0.004016  0.000281  0.070934
2    0.005493  0.009711  0.004140  0.103523
3    0.010577  0.011736  0.010724  0.074775
4    0.013220  0.013055  0.012844  0.069224
..        ...       ...       ...       ...
357  0.996343  0.977458  0.963766  0.205318
358  0.961961  0.955629  0.967307  0.153570
359  0.947145  0.929170  0.828756  0.244820
360  0.867254  0.865348  0.858543  0.151496
361  0.883922  0.861901  0.903027  0.158744

[362 rows x 4 columns]


Repeating this process for the y values:

In [19]:
# define min max scaler
min_max_scaler = preprocessing.MinMaxScaler()

# transform data
y_scaled = min_max_scaler.fit_transform(y)

target = pd.DataFrame(y_scaled)

target.to_csv(r'C:\Users\lynst\Documents\GitHub\machine-learning-projects\machine-learning\target.csv', index = False, header = True)

print(target)

            0
0    0.000000
1    0.004759
2    0.009841
3    0.012585
4    0.012969
..        ...
357  0.963054
358  0.947131
359  0.867923
360  0.859522
361  0.883716

[362 rows x 1 columns]


In [55]:
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

# The training set
bitcoin_features = pd.DataFrame([
    [0.0,0.000638145,0.005493394,0.010576522,0.013219937,0.013616185,0.015992526,0.017530024,0.017145363,0.035601062,
     0.034117637,0.041946603,0.044284035,0.040849555,0.040532128,0.041922571,0.048643045,0.059581053,0.058299042,0.05330532,
     0.03540723,0.033700218,0.038353767,0.047864712,0.056400057,0.04937503,0.050180972,0.056030417,0.054965984,0.054898179,
     0.050392543,0.0421463,0.044964951,0.045510257,0.036752183,0.039062578,0.035227559,0.041737607,0.048730448,0.046944187,
     0.052090113,0.047488634,0.058487009,0.045220009,0.047376197,0.050277531,0.046546652,0.046318058,0.04800018,0.047907054,
     0.049025417,0.050574216,0.042792742,0.045280233,0.045218435,0.044115379,0.044027261,0.045546305,0.04513561,0.043970184,
     0.041858914,0.042636961,0.042154597,0.047263474,0.047587624,0.042780583,0.041644625,0.04048206,0.038145058,0.039819025,
     0.040439861,0.038588512,0.040467183,0.03811001,0.037071613,0.037882991,0.036858039,0.042057896,0.041068707,0.04330157,
     0.041134224,0.041413314,0.040613809,0.04143706,0.04084555,0.040786756,0.038593089,0.038274088,0.038765321,0.038888058,
     0.039383296,0.038263074,0.041289862,0.043857747,0.044787856,0.044080618,0.04675436,0.051075033,0.071235597,0.069841721,
     0.072819157,0.074138791,0.078207267,0.086575674,0.072648785,0.076483089,0.074101169,0.083950569,0.085415112,0.083186254,
     0.085723527,0.084357974,0.087962111,0.078131594,0.080527247,0.083660178,0.084342811,0.0862026,0.086506295,0.092721231,
     0.086650775,0.083285817,0.084835188,0.079343225,0.081119043,0.080912623,0.083593517,0.075068471,0.076896074,0.073581612,
     0.077236389,0.07663043,0.080158179,0.078823811,0.084414336,0.073873434,0.053283577,0.057427297,0.050974612,0.053294878,
     0.055231484,0.052916797,0.053690982,0.056625074,0.057213152,0.058014945,0.055765775,0.06225222,0.06477204,0.067897819,
     0.067002328,0.067521026,0.070586581,0.067439488,0.060011061,0.06146373,0.056849519,0.066242448,0.065901275,0.066912779,
     0.067176276,0.065691564,0.068637958,0.066321268,0.062919404,0.062505704,0.062203154,0.064225734,0.065824886,0.063352844,
     0.063513489,0.068505494,0.072959632,0.075833071,0.075960242,0.078654297,0.074652482,0.075291485,0.077741783,0.082610193,
     0.084569402,0.101885996,0.104740981,0.103790988,0.107104877,0.106081071,0.107993502,0.118650275,0.113695177,0.116947554,
     0.119121337,0.123589065,0.123690917,0.117257972,0.122475852,0.126385973,0.152573221,0.151834369,0.138154527,0.1493629,
     0.146307072,0.146030128,0.154298114,0.166888775,0.167440947,0.162752064,0.160500033,0.173103998,0.191797016,0.194337435,
     0.194577473,0.209741884,0.210133554,0.20483156,0.204466068,0.216216742,0.209358511,0.180506819,0.179016385,0.190291561,
     0.198608041,0.22588404,0.208854118,0.215884438,0.218966014,0.202932147,0.211267652,0.214661363,0.212488724,0.196873849,
     0.201440281,0.193875814,0.19071227,0.204572497,0.210197068,0.212334517,0.213753999,0.249551468,0.276378863,0.28417221,
     0.297506587,0.291165623,0.280067113,0.299595255,0.287985629,0.297663512,0.315458178,0.347915437,0.343747255,0.358541597,
     0.362842958,0.387100466,0.38897084,0.39593221,0.446069266,0.458187005,0.445650417,0.477768505,0.528356883,0.575652543,
     0.601374736,0.591666526,0.559139173,0.509976142,0.477502862,0.538958867,0.569431455,0.531897792,0.519908654,0.514895335,
     0.529266106,0.517064826,0.503196016,0.418617602,0.461838349,0.445054901,0.448768042,0.450626543,0.452262746,0.419173207,
     0.490878723,0.487801295,0.487490448,0.467387103,0.477362673,0.510561788,0.546361832,0.538557756,0.557294976,0.577593727,
     0.570811169,0.702623588,0.705106216,0.676513444,0.730793934,0.723600824,0.716826134,0.744919519,0.727615799,0.755825627,
     0.808243038,0.798422677,0.869950096,0.87282096,0.898043195,0.83887484,0.740842031,0.75075566,0.712325934,0.705474998,
     0.702655918,0.681227218,0.758125294,0.736153148,0.777005564,0.741169328,0.746140019,0.746492207,0.785701411,0.807569989,
     0.852788425,0.871399475,0.897866099,0.884363067,0.953589388,0.918179441,0.857718918,0.872492947,0.904663534,0.895050881,
     0.90612679,0.905726393,0.891463195,0.837382689,0.846340317,0.809868512,0.792719858,0.85578317,0.87089122,0.868240795,
     0.901147373,0.925012638,0.920113759,0.921843087,0.929453473,0.897207928,0.91712402,0.920777509,0.907379905,0.873386578,
     0.909210226,0.905097976,0.933640394,0.939971916,0.937276431,1.0,0.990400651,0.996343364,0.961960942,0.947144571,0.867253609,
     0.883921616],
    [0.0,0.00401551,0.009711295,0.011735948,0.013055226,0.014221307,0.015407219,0.015204028,0.034583606,0.045621022,0.039838514,
     0.042021809,0.045220644,0.038819484,0.040522243,0.04887013,0.05769632,0.057156431,0.056676173,0.050445667,0.03912085,
     0.037919716,0.046090666,0.055120746,0.0538125,0.051038204,0.056136843,0.057359902,0.053678156,0.053435584,0.048343787,
     0.043242215,0.044513593,0.044237923,0.037441692,0.037261823,0.039766314,0.046235763,0.046899103,0.049248443,0.049177501,
     0.056014928,0.054509775,0.045079876,0.048801701,0.048360964,0.045795305,0.04547383,0.04567744,0.04621691,0.048013234,
     0.04881008,0.043215542,0.042761538,0.042554018,0.042709867,0.043676388,0.043023941,0.042406686,0.041286689,0.041161562
     ,0.04127831,0.044934919,0.045083367,0.045788044,0.040548497,0.039963501,0.038604981,0.038408772,0.038850766,0.03842944,
     0.039358395,0.038480971,0.036971629,0.036385795,0.035986395,0.039876499,0.039484919,0.041016885,0.040629075,0.038938885,
     0.039053957,0.03945629,0.038975474,0.039104092,0.038798397,0.036491092,0.036644986,0.037668624,0.037062541,0.03757436,
     0.039434086,0.041194939,0.041825182,0.042897977,0.044419608,0.050400699,0.073931248,0.071441419,0.073119739,0.071963712,
     0.076315221,0.083513366,0.087996566,0.076880247,0.075627722,0.08102242,0.083742951,0.084327528,0.083179881,0.08329188,
     0.087596188,0.084853731,0.079669207,0.08043114,0.087476787,0.08426301,0.083738622,0.090487093,0.089925279,0.083636398,
     0.081723466,0.081416514,0.077690639,0.078189889,0.079492549,0.080078662,0.074132205,0.074644024,0.073805004,0.074537191,
     0.076748836,0.077264565,0.082734815,0.080729015,0.071176782,0.057620769,0.05564276,0.051531728,0.052758139,0.055394182,
     0.052755905,0.055357174,0.05463169,0.057283653,0.057253907,0.061273607,0.063519047,0.066439139,0.066132466,0.065432259,
     0.067934796,0.06722984,0.064768081,0.058974681,0.060512232,0.063381491,0.06377377,0.06412122,0.064611672,0.067053461,
     0.065588388,0.06548784,0.065191222,0.060620462,0.059903915,0.06132025,0.062659498,0.062641483,0.060572561,0.064368541,
     0.072215502,0.071978096,0.074197422,0.076184648,0.075762205,0.072380848,0.073999956,0.079323712,0.082396023,0.104702395,
     0.104015035,0.101210854,0.103558658,0.106933451,0.106471627,0.116222303,0.117483068,0.116485824,0.116658013,0.123571829,
     0.120480526,0.12012721,0.118519973,0.123063921,0.149197834,0.153152177,0.149670132,0.147053919,0.149337065,0.14341756,
     0.152887818,0.161796402,0.165139077,0.161782717,0.158213111,0.170087033,0.187860071,0.198952649,0.193923137,0.205003984,
     0.208843395,0.204315368,0.203590582,0.214723519,0.215150849,0.205282727,0.179889659,0.186527107,0.194575584,0.219270259,
     0.222169264,0.211593532,0.21450301,0.213110555,0.204679855,0.208725251,0.20909728,0.206962443,0.196297195,0.194974147,
     0.18764878,0.200043599,0.208283257,0.206557317,0.208663386,0.244457202,0.282553297,0.276036647,0.292759793,0.29513818,
     0.294244836,0.290939448,0.294771597,0.289880618,0.306877619,0.342979505,0.371220883,0.353430249,0.352649603,0.377962231,
     0.38350063,0.38883472,0.45202489,0.477863582,0.455673678,0.472075628,0.514932966,0.575590296,0.606549552,0.596875544,
     0.59658926,0.544173265,0.513570117,0.529706592,0.568843362,0.561499561,0.536126883,0.515710959,0.528453647,0.533958949,
     0.508835547,0.489317855,0.463331692,0.455857039,0.448442157,0.480880731,0.443877681,0.439836475,0.43418468,0.547374333,
     0.484143665,0.474538504,0.481677577,0.504755239,0.531548023,0.551563289,0.545843904,0.590642244,0.56834872,0.684585128,
     0.716206886,0.698926674,0.72199093,0.727419005,0.714662771,0.740197164,0.728657006,0.751042281,0.794336447,0.792902655,
     0.851649984,0.875727823,0.890268651,0.875303146,0.81726175,0.763639593,0.767996549,0.720534515,0.720908499,0.693572056,
     0.741592273,0.747704635,0.789266575,0.776934878,0.736188078,0.7311129,0.769060825,0.787397493,0.83084667,0.872854375,
     0.879124961,0.877294143,0.937312572,0.935815099,0.917421594,0.850059505,0.884007142,0.907187866,0.901586625,0.912633817,
     0.890535384,0.882197132,0.844605316,0.868478567,0.800047174,0.833763829,0.858966832,0.859514541,0.888210762,0.910180716,
     0.917834121,0.909353985,0.917482063,0.918206569,0.897194478,0.909424928,0.902699639,0.894637337,0.886139326,0.896831526,
     0.934804728,0.926298896,0.93701735,0.978355012,1.0,0.98054138,0.977457618,0.955629409,0.929169971,0.865348161,0.861900889],
    [0.0,0.00028148,0.004139987,0.010723582,0.012843945,0.013667648,0.016291556,0.016095374,0.016633626,0.031952063,0.034822303,
     0.041763704,0.042175041,0.03653427,0.040065414,0.042270338,0.046896517,0.057111407,0.054434408,0.029994941,0.029660224,
     0.033651236,0.039280241,0.04889717,0.049099677,0.049069235,0.050912677,0.054973837,0.05265538,0.050189859,0.038844639,
     0.042380636,0.044698211,0.037692249,0.036190877,0.034912012,0.036014695,0.042158423,0.046433414,0.046649745,0.048104645,
     0.045626917,0.044697181,0.04501778,0.046138404,0.047529773,0.046244731,0.043712297,0.047463006,0.048459068,0.048655692,
     0.042446079,0.043290665,0.04470777,0.044279374,0.037681072,0.044093927,0.042980214,0.043499937,0.042550347,0.042011506,
     0.043178015,0.043238605,0.0470777,0.042530494,0.0394451,0.040444985,0.038054319,0.03759754,0.038872728,0.03973658,
     0.038652721,0.037488566,0.037455182,0.037352238,0.035942339,0.0374393,0.040583813,0.041928415,0.041494872,0.039582016,
     0.040865587,0.040716023,0.041513108,0.040684404,0.039205092,0.037823871,0.038544335,0.038687575,0.038623897,0.039099353,
     0.038867875,0.041526932,0.043931863,0.043967011,0.044956749,0.047376239,0.052097862,0.066339499,0.070514928,0.071871443,
     0.073844007,0.078617102,0.074247402,0.074047249,0.075836865,0.075154196,0.083179897,0.081647494,0.084457732,0.084267873,
     0.086575301,0.075996282,0.077440594,0.075231846,0.084815244,0.086559712,0.085506443,0.086975608,0.088692575,0.084164341,
     0.084671857,0.081022621,0.078771666,0.08015024,0.082247073,0.074021954,0.075727891,0.072804708,0.075390821,0.077813694,
     0.07831621,0.079880526,0.079349185,0.073885332,0.05367306,0.053893361,0.048018318,0.050135593,0.047909491,0.04966543,
     0.05189815,0.054966631,0.053460259,0.056392119,0.055068693,0.056030489,0.063185132,0.064671504,0.066482445,0.067035845,
     0.068800461,0.066952901,0.060062826,0.060390924,0.057447888,0.057964081,0.065436088,0.067245558,0.066485092,0.067448506,
     0.067113789,0.067628217,0.062058331,0.061433752,0.063017039,0.063082776,0.0649171,0.06372265,0.064422084,0.06268394,
     0.070243596,0.0747133,0.07566333,0.076355411,0.075094635,0.075883926,0.077091318,0.078192383,0.083561821,0.086511769,
     0.103225693,0.103537614,0.105669007,0.106261673,0.106174317,0.110583578,0.110117386,0.111988182,0.114671504,0.120570518,
     0.123917391,0.115890808,0.116013312,0.122133069,0.129089176,0.148814373,0.133994776,0.14015924,0.140616755,0.146422678,
     0.149917939,0.15541341,0.166638627,0.161010944,0.162093627,0.163412346,0.175346555,0.190580283,0.192149452,0.198652015,
     0.212070096,0.197186968,0.20302598,0.204730447,0.212295104,0.169770787,0.172460138,0.180025089,0.192414608,0.20403116,
     0.206576536,0.206147699,0.216687452,0.208520276,0.206442561,0.212227601,0.213501907,0.201261217,0.194405554,0.192794914,
     0.187922845,0.195875014,0.208802638,0.213780299,0.215145196,0.217475418,0.254924424,0.277374121,0.286233819,0.292275338,
     0.276672922,0.281702496,0.287866519,0.287093259,0.300628109,0.321238187,0.346799455,0.351935651,0.347132995,0.372874417,
     0.384544941,0.396170317,0.401556668,0.456992873,0.392238867,0.423707385,0.481437782,0.53748386,0.543909509,0.5843567,
     0.528726665,0.430848057,0.470705817,0.466356706,0.545017339,0.506538888,0.524417555,0.493463612,0.512639968,0.531792205,
     0.480735553,0.419243564,0.395853984,0.44669607,0.439756139,0.458345417,0.436818691,0.410307539,0.423154867,0.463856037,
     0.475865102,0.46346573,0.466085667,0.489864525,0.523342961,0.542537553,0.547852725,0.572782649,0.559327066,0.571676584,
     0.702506993,0.676720717,0.682914741,0.726313793,0.723302666,0.736790161,0.720922736,0.733307548,0.774673004,0.809088971,
     0.808952937,0.870424926,0.889815994,0.76749836,0.697313001,0.730935118,0.730557018,0.682239572,0.704895747,0.666919077,
     0.699298654,0.733420934,0.753969834,0.744817185,0.72548053,0.736375294,0.767288648,0.779661695,0.825237434,0.85033457,
     0.866252585,0.877150437,0.888400654,0.944433055,0.872755883,0.839719079,0.856840515,0.84976161,0.897998612,0.929211092,
     0.888997438,0.856748012,0.843995247,0.828701958,0.798304945,0.812901594,0.863051746,0.875052575,0.88045275,0.916693334,
     0.926739836,0.940617344,0.942197248,0.922215719,0.914167223,0.924618591,0.920758613,0.88977011,0.893540673,0.927637218,
     0.926381883,0.949434982,0.955569004,0.963515143,0.990582636,1.0,0.963765739,0.967307031,0.82875593,0.858542776,0.903026716],
    [0.068801303,0.070934219,0.103522793,0.074775102,0.069223858,0.069766106,0.079274715,0.069174138,0.155427915,0.178482378,
     0.107393372,0.094354869,0.116035118,0.110940503,0.102128676,0.123828751,0.159748802,0.129982635,0.11300985,0.166255267,
     0.147451312,0.099227896,0.110503201,0.145339655,0.119238149,0.079890324,0.092590675,0.096830865,0.088474634,0.079413152,
     0.088783989,0.058331273,0.051554194,0.067009076,0.06291884,0.056086027,0.06599643,0.071333378,0.06661354,0.066057767,
     0.050245305,0.072449773,0.08418904,0.03975008,0.042666361,0.034656687,0.025078483,0.039196795,0.028082052,0.03530522,
     0.04154569,0.057173929,0.032732003,0.016798021,0.021527396,0.045305106,0.029151112,0.024994477,0.017465898,0.023419532,
     0.015508525,0.009823935,0.027551227,0.014854065,0.021454824,0.020326578,0.019669376,0.016271573,0.007614992,0.013573907,
     0.010972728,0.011779643,0.012855415,0.002517949,0.000035430,0.001966808,0.017617459,0.005089285,0.023225449,0.018139752,
     0.014598851,0.003173169,0.007000394,0.016740756,0.018489717,0.011067529,0.010892915,0.005341423,0.0,0.002149397,
     0.004568928,0.017787133,0.012883929,0.017858239,0.012943928,0.013114862,0.025266747,0.070946585,0.050765872,0.037650791,
     0.032569575,0.033512601,0.042597316,0.046671214,0.024434458,0.026996417,0.036582132,0.033762203,0.033285371,0.015990107,
     0.015751147,0.042370452,0.044966588,0.038469633,0.04589681,0.03605968,0.033337394,0.02472389,0.048063053,0.040951633,
     0.036564873,0.023098188,0.034086521,0.023254093,0.017945027,0.02485531,0.041801771,0.029940013,0.032202425,0.02162331,
     0.014557348,0.021396463,0.028856986,0.044181557,0.046332342,0.058820454,0.052292305,0.09767545,0.073976354,0.063892159,
     0.064184132,0.035077684,0.128082493,0.099741261,0.073864463,0.073166798,0.069888873,0.061000531,0.055583094,0.077914947,
     0.042164248,0.031187529,0.037086735,0.050616218,0.034324217,0.032844008,0.032743114,0.027374079,0.017658023,0.017358158,
     0.031861761,0.02496948,0.025538994,0.045246558,0.032868403,0.014206562,0.015536422,0.021061674,0.030254725,0.015669027,
     0.028660048,0.031073084,0.022268907,0.034997376,0.036605971,0.039921314,0.019980407,0.017338668,0.034467443,0.055587116,
     0.094043855,0.067347455,0.049708957,0.036189621,0.03590427,0.051743512,0.064790338,0.072277859,0.136149918,0.055996165,
     0.055144892,0.037300932,0.055869309,0.05228325,0.068449047,0.085545722,0.082250015,0.067642279,0.041928704,0.064562967,
     0.03877925,0.051679549,0.065671092,0.057735237,0.045173428,0.033462862,0.05713098,0.08012656,0.110562162,0.073820384,
     0.073878545,0.081962824,0.086848031,0.091186356,0.116794238,0.093398972,0.147102843,0.078725892,0.059751637,0.055264995,
     0.105425347,0.110505239,0.073559814,0.056812887,0.06195675,0.042262184,0.036460422,0.041356578,0.055718143,0.063934562,
     0.036973671,0.044176682,0.02589003,0.036746087,0.027977903,0.040249128,0.092880322,0.172513636,0.081352235,0.075705225,
     0.073988518,0.098275306,0.093666368,0.113982358,0.084114637,0.087187097,0.105926972,0.15970267,0.10773841,0.096207392,
     0.113305949,0.099637376,0.081815867,0.162082513,0.193700111,0.202994634,0.160298739,0.182919726,0.212289158,0.221583681,
     0.144095994,0.198347373,0.328470698,0.182127416,0.165995311,0.148296904,0.161886916,0.132128647,0.116722056,0.108015522,
     0.130561378,0.157459784,0.183667846,0.18978203,0.10441161,0.105188663,0.138627377,0.139054842,0.147724077,0.196023742,
     0.312205282,0.154774648,0.118258642,0.14466324,0.148794118,0.143049089,0.16648558,0.135119067,0.172865911,0.155549131,
     0.261085404,0.233201835,0.21926005,0.200671004,0.187207337,0.168622423,0.171414691,0.187625047,0.188934642,0.200671004,
     0.114726242,0.147540005,0.161097027,0.113292447,0.230878204,0.272703559,0.146533312,0.121098183,1.0,0.097216527,
     0.119015393,0.119633917,0.100897156,0.118078747,0.115555487,0.10431407,0.062377789,0.087954552,0.104284583,0.110946077,
     0.129371817,0.126708074,0.122776818,0.137216549,0.08853972,0.15391357,0.134096113,0.13479744,0.123195911,0.104130755,
     0.107907993,0.112500645,0.125786173,0.126438872,0.167621242,0.160465462,0.127366458,0.09986011,0.100791711,0.129931932,
     0.120927475,0.152629522,0.141152479,0.132937827,0.135606996,0.109502459,0.137948778,0.154302602,0.183081892,0.116213728,
     0.097169773,0.130893472,0.096130813,0.112632721,0.165167693,0.186609154,0.139042109,0.205318265,0.153570233,0.244819989,
     0.15149584,0.1587442] 
    ])

target = pd.Series([
    [0.0,0.004758968,0.009840699,0.012585046,0.01296924,0.015361182,0.016863592,0.016513049,0.035047592,0.033211393,0.041244961,
     0.043774085,0.040269368,0.039913097,0.041220618,0.048175206,0.059292352,0.05773696,0.052757757,0.034804445,0.032905383,
     0.037717763,0.047229685,0.055789354,0.048666654,0.049617474,0.055330842,0.054355535,0.05437143,0.049791886,0.041560994,
     0.044304195,0.044839747,0.036218217,0.038392502,0.034560726,0.041046062,0.048072105,0.046339437,0.051492622,0.046830025,
     0.057973519,0.044532736,0.046784346,0.049680194,0.045949371,0.045721547,0.047369014,0.047244147,0.048448997,0.049969306,
     0.042200793,0.044671779,0.04456338,0.043518337,0.043346216,0.045018169,0.044499371,0.043388315,0.04118754,0.042051296,
     0.041593643,.046744681,0.046931122,0.042157261,0.041104772,0.039770474,0.037467027,0.03927244,0.039927417,0.037815709,
     0.039796966,0.037464307,0.036512198,0.037388986,0.036260174,0.041954067,0.040439342,0.042695534,0.040609029,0.040823966,
     0.039984552,0.040808787,0.040326217,0.040197198,0.037992269,0.037664351,0.038152219,0.038291119,0.038739465,0.037678528,
     0.040671462,0.043211468,0.044091406,0.043418959,0.046085264,0.050469489,0.070143584,0.069253622,0.072248848,0.073576989,
     0.077647471,0.086023134,0.072255005,0.075907213,0.073559375,0.083325326,0.084862246,0.082555363,0.085474838,0.084045744,
     0.087327505,0.077667519,0.07988691,0.083306138,0.083775391,0.085615027,0.085883662,0.092206614,0.086089004,0.082653309,
     0.084271421,0.078899718,0.080585132,0.080360315,0.083041083,0.074473967,0.076369592,0.072964111,0.076685482,0.076017188,
     0.079549541,0.078275528,0.083949373,0.073419903,0.052962097,0.056815638,0.050413786,0.052680288,0.054635913,0.052264018,
     0.053089255,0.055914794,0.056445907,0.057224319,0.055071944,0.061717229,0.06416602,0.067341529,0.066360638,0.067138048,
     0.069973038,0.066914806,0.059465762,0.060850609,0.056219801,0.06561645,0.065312732,0.066352906,0.066641445,0.065271348,
     0.068116648,0.065781841,0.062306623,0.061938467,0.06156358,0.063565887,0.065315452,0.062796209,0.062950145,0.066541494,
     0.072382593,0.074257168,0.07543567,0.077936154,0.074071873,0.074751623,0.076943807,0.081963391,0.084060064,0.101748606,
     0.104084702,0.103239561,0.106557263,0.105531982,0.107447082,0.118114593,0.113154292,0.116410277,0.11858585,0.123058427,
     0.123161814,0.116721585,0.12194064,0.125857761,0.152074327,0.151332573,0.137638762,0.14885815,0.145798916,0.145522405,
     0.153799692,0.16640049,0.166955087,0.162261416,0.160007229,0.173193553,0.191340036,0.193881761,0.194119466,0.209299075,
     0.209691289,0.204383595,0.204001262,0.215837537,0.20895784,0.179973013,0.178458002,0.189787794,0.198138398,0.225310223,
     0.208433743,0.215358547,0.218503842,0.202507873,
     0.210827404,0.214260092,0.212050725,0.196422769,0.201000451,0.193436422,0.19038893,0.204067705,0.209716348,
     0.211889343,0.213283784,0.24919055,0.275974458,0.283898911,0.297301175,0.290858225,0.279862652,0.299263673,
     0.287643767,0.297354587,0.314898073,0.347570763,0.343284627,0.358306006,0.362599587,0.386827594,0.388851666,
     0.395637713,0.445828972,0.457471074,0.445354851,0.477214905,0.528067013,0.575388773,0.601495078,0.591619869,
     0.559233857,0.510760087,0.477476381,0.538693284,0.569910676,0.531851964,0.520046905,0.514746801,0.528927763,
     0.517294826,0.503023077,0.418534429,0.462017826,0.444911947,0.44863475,0.450571759,0.452173404,0.418788458,
     0.441684171,0.488016734,0.487306483,0.467203517,0.477266456,0.510416561,0.546207768,0.538350187,0.557315463,
     0.577815941,0.571040777,0.702893571,0.705375583,0.676901536,0.731470719,0.723931176,0.716737737,0.745400516,
     0.727712833,0.755805836,0.808557579,0.798650437,0.870196051,0.873614563,0.898433395,0.839117845,0.740749375,
     0.750798281,0.710832058,0.705447181,0.702615055,0.680870922,0.758588989,0.735581726,0.777431403,0.741889213,
     0.74675386,0.746483506,0.786436125,0.807258507,0.853004397,0.872452386,0.897837128,0.884407657,0.954298644,
     0.919083083,0.859124584,0.872354154,0.904536971,0.895459935,0.906653262,0.906061147,0.891815747,0.837894381,
     0.847060771,0.810894677,0.793254248,0.855995614,0.871097754,0.868521806,0.901411867,0.925072819,0.920257432,
     0.922072438,0.929509882,0.89745007,0.91736187,0.919016211,0.907754867,0.872700831,0.909434839,0.905207987,
     0.932986103,0.940795713,0.93763839,1.0,0.991373458,0.997701278,0.963054205,0.947131409,0.867923391,
     0.859522382,0.883715735]])

X = bitcoin_features[['Open'],['High'],['Low'],['Volume']]
y = target[['Close']]


fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(bitcoin_features['Open'], target['Close'], bitcoin_features['Volume'], c='b')
ax.set_xlabel("Open")
ax.set_ylabel("Close")
ax.set_zlabel("Volume")
plt.show()


'''
# 5 degree polynomial features
deg_of_poly = 5
poly = PolynomialFeatures(degree=deg_of_poly)
X_ = poly.fit_transform(X)

# Fit linear model
clf = linear_model.LinearRegression()
clf.fit(X_, y)


# The test set, or plotting set
N = 20
Length = 1
predict_x0, predict_x1 = np.meshgrid(np.linspace(0, Length, N), 
                                     np.linspace(0, Length, N))
predict_x = np.concatenate((predict_x0.reshape(-1, 1), 
                            predict_x1.reshape(-1, 1)), 
                           axis=1)
predict_x_ = poly.fit_transform(predict_x)
predict_y = clf.predict(predict_x_)


# Plot
fig = plt.figure(figsize=(16, 6))
ax1 = fig.add_subplot(1, projection='3d')
surf = ax1.plot_surface(predict_x0, predict_x1, predict_y.reshape(predict_x0.shape), 
                        rstride=1, cstride=1, cmap=cm.jet, alpha=0.5)
ax1.scatter(bitcoin_features[:, 0], bitcoin_features[:, 1], bitcoin_features[:, 2], bitcoin_features[:, 3], c='b', marker='o')

ax1.set_xlim((1, 0))
ax1.set_ylim((0, 1))
fig.colorbar(surf, ax=ax1)
ax2 = fig.add_subplot(1)
cs = ax2.contourf(predict_x0, predict_x1, predict_y.reshape(predict_x0.shape))
ax2.contour(cs, colors='k')
fig.colorbar(cs, ax=ax2)
plt.show()
'''

TypeError: 'type' object is not subscriptable

## Split the Dataset
First I've decided to import the train_test_split function to split the dataset into respective training and test sets (for both X and y values in each set).

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Next, a linear regression model is applied and visualized to see how well the prediction line fits.

## Train Linear Regression Model
Train the linear model on the data and fit a regression line.

In [ ]:
# Fitting Linear Regression to the dataset
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Visualizing the Linear Regression results
def viz_linear():
    plt.scatter(X, y, color='red')
    plt.plot(X, lin_reg.predict(X), color='blue')
    plt.title('Truth or Bluff (Linear Regression)')
    plt.xlabel('Position level')
    plt.ylabel('Salary')
    plt.show()
    return
viz_linear()


'''
# Fitting Linear Regression to the dataset
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
'''


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

pf = PolynomialFeatures(degree=2)
poly_X = pf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.3, random_state=0)

from sklearn.linear_model import Ridge
reg_regression = Ridge(alpha=0.1, normalize=True)
reg_regression.fit(X_train, y_train)
print('R2: %0.3f' %r2_score(y_test,reg_regression.predict(X_test)))

## Predicting Against Test Set
Now to predict the test set results.

In [ ]:
y_pred = lin_reg.predict(X_test)

# to see the first ten results
y_pred[0:10]

Comparing the first row of X values to predict the y value by copying from the dataset output from cell 1. Remember only to use the 'Open', 'High', 'Low' and 'Volume' values.

In [ ]:
lin_reg.predict([[9718.07, 9838.33, 9728.25, 4.624843e+10]])

So the predicted y value is 9777.86 and the actual y value (from output of cell 1 for 'Close' price) is 9763.94, which is reasonably similar.

## Model Evaluation
This is where I need to look into the root mean squared error and R-squared values to see how well the model is performing.

In [ ]:
# import the score measure
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

## Visualization
Plotting the actual and predicted results for the 'Close' price and fitting a regression line provides us with a nice visual display.

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(y_test, y_pred)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs. Predicted Prices")

## Predictions
Storing the actual and predicted values and their difference in a dictionary object becomes useful. The larger the error, the greater the variance and subsequent standard deviation which translates to less accurate predictions and a less reliable model. 

Why does this become important? The main purpose of this exercise is to see how useful my model would be in predicting the daily closing price so the first information I can derive from the scatter plot above shows the relationship between independent and dependent variables is strong, but also incredibly linear. It may not pay off to apply Polynomial Regression in such circumstances but may improve the linear fit to the data points with increasing degrees of freedom.

In [ ]:
predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_pred, "Error": y_test - y_pred})
predictions[0:20]

Exploring the entire dataset (all the X and y values) and the degree of accuracy by fitting a linear regression predictive line once more:

In [ ]:
# converting the dictionary value objects
X = predictions[y_test].values
y = predictions[y_pred].values

# Fitting Linear Regression to the dataset
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)

# Visualizing the Linear Regression results
import matplotlib.pyplot as plt
def viz_linear():
    plt.scatter(X, y, color='red')
    plt.plot(X, lin_reg.predict(X, color='blue')
    plt.title('Actual vs. Predicted Bitcoin Prices')
    plt.xlabel('Actual Price')
    plt.ylabel('Predicted Price')
    plt.show()
    return
viz_linear()

In [ ]:
import seaborn as sns

# set the width and height of the plot
plt.figure(figsize=(7,7))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, price_predictions)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=price_predictions)

In [ ]:

'''
# Visualising the Linear Regression results
plt.scatter(X, y, color = 'blue')
  
plt.plot(X, lin.predict(X), color = 'red')
plt.title('Linear Regression')
plt.xlabel('Temperature')
plt.ylabel('Pressure')
  
plt.show()'''

## Training the Polynomial model
Importing the relevant libraries and instantiating the polynomial function model followed by splitting the data into training and validation sets. The first stage will produce an expanded feature set for the entire dataframe with quadratic terms (degree=2). I will attempt to tune this hyperparameter later to see if it can optimize the Loss function.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

# initialize the model for a given degree
poly_features = PolynomialFeatures(degree=2, include_bias=True)
  
# transforms the existing features to higher degree features.
poly_X = poly_features.fit_transform(X)

Using indexation to return any value in X, say the 1st value

In [ ]:
X[:1]

Now repeating for the data contained in the 'poly_X' set and we can see that there are 15 values returned so it has created an array with 11 new features for a total of 15 features. This includes element-wise dot product values and some squared values (without going into too much detail). Both the original feature values for 'X1' to 'Xn' and the feature squared value from 'poly_X' are returned in this example.

In [ ]:
poly_X[:1]

Saving the current dataframe to a CSV formatted file (or an Excel file) for preservation to view the degree of newly expanded features in a fresh table.

In [ ]:
df.to_csv(r'C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/X_train_poly.csv', index=False, header=True)

### Splitting the Data
Using this data for the Polynomial Features model and splitting it into training and test sets with a 70-30 split. 

In [ ]:
# then split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size = 0.3, random_state = 42)

The number of entries (rows) by the number of features (cols):

In [ ]:
print(X.shape)
print(y.shape)

But the shape of the new dataset with all the extended features is much larger.

In [ ]:
print(poly_X.shape)

Printing out the shape of the training set from the polynomial data gives:

In [ ]:
print(X_train.shape)
print(y_train.shape)

And the shape of the test data:

In [ ]:
print(X_test.shape)
print(y_test.shape)

### Applying Linear Regression First

In [ ]:
from sklearn.linear_model import LinearRegression

# fit the transformed features to Linear Regression
linear_regression = LinearRegression()

# fit the model to training set only!!!
linear_regression.fit(poly_X, y)

### Making a Prediction
Comparing actual and predicted 'Close' prices between labelled and target data in the test set tells me that the accuracy isn't great and there seems to be considerable variance.

In [ ]:
poly_y_pred = linear_regression.predict(poly_X)
#predictions = pd.DataFrame(poly_y_pred)

#print("Predictions: ", linear_regression.predict(poly_X.iloc[:4]))
#print("Actual: ", poly_X.iloc[:1])


In [ ]:
print(y.shape)

Comparing predictions for training set and test set values

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns

mse = mean_squared_error(y_test, y_test_pred)
print(mse)

rmse = np.sqrt(mse)
print(rmse)

# set the width and height of the plot
plt.figure(figsize=(6,6))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, y_test_pred)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=y_test_pred)

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)
  
# evaluating the model on test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

print("\n")

print("The model performance for the training set")
print("-------------------------------------------")
print("RMSE of training set is {}".format(rmse_train))
print("R2 score of training set is {}".format(r2_train))
  
print("\n")
  
print("The model performance for the test set")
print("-------------------------------------------")
print("RMSE of test set is {}".format(rmse_test))
print("R2 score of test set is {}".format(r2_test))

print("\n")

Defining the intercept of the polynomial line and coefficient values for x1, x2 ... xn.

In [ ]:
ridge_regression.intercept_
ridge_regression.coef_

Given:

y = a + b1x1 + b2x2 + b3x3 + b4x4

### Utilizing Regularization
In this particular instance I have chosen L2 regularization which will penalize the Loss (or cost) function by summing the coefficients and equalizing features which exhibit strong colinearity, while simultaneously reducing the effect of redundant coefficients. (See the colinearity matrix in the previous <a href="https://github.com/lynstanford/machine-learning-projects/machine-learning/multiple_regression.ipynb">Linear Regression</a> model).
    
Note, I have two hyperparameters I can adjust here including 'alpha' and 'normalize'. The 'alpha' value is the learning rate which controls the size of the incremental steps for the learning rate and the 'normalize' adjustment will re-scale the feature values between 0 and 1 for each observation in the dataset.

In [ ]:
from sklearn import linear_model

# initialize ridge regression
ridge_regression = linear_model.Ridge(alpha=0.1, normalize=True)

# fit the model to training set only!!!
ridge_regression.fit(X_train, y_train)

The regularization term should only be added to the cost function during the training phase. Now the 'training' data has been fit, it becomes important to discover the performance measure on the unregularized test set. Making a prediction on the first 5 values in the test set first.

In [ ]:
y_test_pred = ridge_regression.predict(X_test)
print("Predictions: ", ridge_regression.predict(X_test.iloc[:5]))

Defining the intercept of the polynomial line and coefficient values for x1, x2 ... xn.

In [ ]:
ridge_regression.intercept_
ridge_regression.coef_

Given:

    y = a + b1x1 + b2x2 + b3x3 + b4x4

This tells me the prediction for the target output variable, y, based on the input variables specified and using ridge regression giving the value of C$37,990.99.

## Model Validation Metrics¶
Now to measure the error score and accuracy of the line of fit.

## Applying Regularization
To avoid overfitting regularization can be applied, but will require tuning hyperparameters manually to an extent. To see if I can improve on the linear regression model's ability to predict the target variable, I have decided to use Ridge Regression.
This regularization term basically employs the use of a penalization method by summing the squared values of each coefficient (whether positive or negative) and helps balance the contribution of all the features but this term is only used during the training phase so I need to remove it for the purpose of testing and evaluation.

## Model Selection
### Ridge Regression
Trying a slightly different type of regression model using the alpha learning rate hyperparameter of 1.0 to see if I can reduce the (rmse) error value and increase the (r2) accuracy score. The purpose of using a ridge regression model is to try to reduce or eliminate the coefficient values of all the various features (especially those with high multi-colinearity between predictors) and will increase bias slightly but should decrease variance significantly. Ridge regression achieves this by assigning equal weights to those coefficient values which have high colinearity.

In [ ]:
# instantiate model
ridge_regression = linear_model.Ridge(alpha=1.0)

# fit model
ridge_regression.fit(X_train, y_train)

The regularization term should only be added to the cost function during the training phase. Now the 'training' data has been fit, it becomes important to discover the performance measure on the unregularized test set. Making a prediction on the first 5 values in the test set first.

In [ ]:
price_predictions = ridge_regression.predict(X_test)
print("Predictions: ", ridge_regression.predict(X_test.iloc[:5]))

This tells me the prediction for the target output variable, y, based on the input variables specified and using ridge regression giving the value of C$37,990.99.

## Model Validation Metrics¶
Now to measure the error score and accuracy of the line of fit.

This new data matrix containing the additional features with the squared values has been created by expanding the number of features and the parameter weights (or coefficients) which represent a quadratic equation. The linear regression model should now be applied again to this newly expanded dataframe containing the polynomial features.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, price_predictions)
print(mse)

rmse = np.sqrt(mse)
print(rmse)

# set the width and height of the plot
plt.figure(figsize=(6,6))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, price_predictions)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=price_predictions)

In [ ]:
print("R-squared: ", ridge_regression.score(X_test, y_test))

Producing a scatter plot of the data points to display actual vs predicted prices and the regression line of best fit.

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, price_predictions)
print(mse)

rmse = np.sqrt(mse)
print(rmse)

# set the width and height of the plot
plt.figure(figsize=(6,6))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, price_predictions)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=price_predictions)

In [ ]:
print(ridge_regression.intercept_)
print(ridge_regression.coef_)

This is a slight improvement on the original R-squared score and I can see that the RMSE hasn't really changed.
Trying to improve on the scores above, I decided to introduce a Polynomial model which should be able to fit the line more accurately to the data points.

## Using a Pipeline
The next method involves placing the expanded polynomial features and linear regression of these within a pipeline which can be trained and used to predict the target variable.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_train, y_train)
y_train_pred = pipeline.predict(X_train)
print(y_train_pred)

Next I wanted to find the shape of the matrices involved and now these datasets have been expanded by the polynomial model I want to make sure they have the same dimensions (m.n) otherwise the polynomial regression won't work.

In [ ]:
print(X_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print(y_train_pred.shape)

In [ ]:
# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

# print the RMSE metric and R2 accuracy score
print("RMSE of training set is {}".format(rmse_train))
print("R2 score of training set is {}".format(r2_train))

The metrics from using the 2nd order quadratic coefficients and terms on the training set, a significant improvement.

In [ ]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_test, y_test)
y_test_pred = pipeline.predict(X_test)
print(y_test_pred)

The metrics from generalizing to the test set data for the quadratic equation.

In [ ]:
# evaluating the model on test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

# print the RMSE metric and R2 accuracy score
print("RMSE of test set is {}".format(rmse_test))
print("R2 score of test set is {}".format(r2_test))

In [ ]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_train, y_train)
y_train_pred = pipeline.predict(X_train)
print(y_train_pred)

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

# print the RMSE metric and R2 accuracy score
print("\n")
print("RMSE of training set is {}".format(rmse_train))
print("R2 score of training set is {}".format(r2_train))

In [ ]:
print(X_test.shape)

In [ ]:
print(y_test.shape)

In [ ]:
print(y_test_pred.shape)

In [ ]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_test, y_test)
y_test_pred = pipeline.predict(X_test)
print(y_test_pred)

# evaluating the model on test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

# print the RMSE metric and R2 accuracy score
print("\n")
print("RMSE of test set is {}".format(rmse_test))
print("R2 score of test set is {}".format(r2_test))

The metrics from using an equation with 3rd order cubic coefficients and terms on the test set produced the best overall scores so far.

# Model Validation
So the fit of the regression forecast line to the data has increased in its accuracy marginally with the degree of fit (or expansion of terms) and the RMSE has reduced fairly considerably. The R-squared accuracy appears to increase with smaller sets of data such as the test sets overall.

Ridge regression rmse score = 689.1925582115692
R-squared accuracy = 0.9991392014478754

So I can determine the use of ridge regression to regularize the model has produced a slight increase in overall variance but also a slight increase in bias of output prediction. Ridge regression should reduce the variance and if not, I probably need to adjust the 'degrees of freedom' hyperparameter for the cost function for the Polynomial model which feeds into it, or tune the 'alpha' hyperparameter in the actual ridge regression model itself. Tuning these inputs ...........

I have decided to see if I can improve the model's predictive power by electing to use a Decision Tree Regression model: "https://github.com/lynstanford/machine-learning-projects/tree/master/machine-learning/decision_tree.ipynb".

# Print dependencies
Dependencies are fundamental to record the **computational environment**.   

- Use [watermark](https://github.com/rasbt/watermark) to print version of python, ipython, and packages, and characteristics of the computer

In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,watermark,tarfile,urllib3,matplotlib,seaborn,sklearn,pickle5

# date
print (" ")
%watermark -u -n -t -z 